# Densenet Approach

In [5]:
%pip install torch
%pip install torchsummary
%pip install torchvision
%pip install scikit-learn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
#import shutil
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
#import matplotlib.pyplot as plt
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
#from torchsummary import summary
from torchvision import datasets, transforms, models

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available:
    print('lovin the GPU')

lovin the GPU


In [8]:
# Define constants
OUTPUT_DIM = 4  # Change this to your number of classes
BATCH_SIZE = 64
NUM_WORKERS = 2
PIN_MEMORY = True
N_EPOCHS = 15

# Define transforms
mean_imagenet = [0.485, 0.456, 0.406]
std_imagenet = [0.229, 0.224, 0.225]

train_transform_imagenet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_imagenet, std_imagenet)
])

test_transform_imagenet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_imagenet, std_imagenet)
])

# Load datasets
train_dataset_imagenet = datasets.ImageFolder(root='Data/archive/grape_dataset/train', transform=train_transform_imagenet)
test_dataset_imagenet = datasets.ImageFolder(root='Data/archive/grape_dataset/test', transform=test_transform_imagenet)

# Create data loaders
train_iterator_imagenet = DataLoader(train_dataset_imagenet, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_iterator_imagenet = DataLoader(test_dataset_imagenet, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

# Define the model
class DenseNet121(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.dense_net = models.densenet121(pretrained=True)
        num_features = self.dense_net.classifier.in_features
        self.dense_net.classifier = nn.Identity()
        self.fc = nn.Linear(num_features, output_dim)

    def forward(self, x):
        x = self.dense_net(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

dense_net121_extract = DenseNet121(OUTPUT_DIM).to(device)

# Define the optimizer (added L2 regularization)
optimizer = optim.Adam(dense_net121_extract.parameters(), weight_decay=0.01)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Training function
def model_training(N_EPOCHS, model, train_iterator, valid_iterator, optimizer, criterion, device, model_name='model.pt'):
    best_valid_loss = float('inf')
    for epoch in range(N_EPOCHS):
        # Training
        model.train()
        train_loss = 0
        valid_loss = 0
        for inputs, labels in train_iterator:
            optimizer.zero_grad()
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        with torch.no_grad():
            for inputs, labels in valid_iterator:
                inputs, labels = inputs.to(device), labels.to(device)
                predictions = model(inputs)
                loss = criterion(predictions, labels)
                valid_loss += loss.item()

        print(f'Epoch: {epoch+1}/{N_EPOCHS}.. Training Loss: {train_loss/len(train_iterator):.3f}.. Validation Loss: {valid_loss/len(valid_iterator):.3f}')

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), model_name)

    return model

# Train the model
model_training(N_EPOCHS, dense_net121_extract, train_iterator_imagenet, test_iterator_imagenet, optimizer, criterion, device, model_name='dense_net121.pt')
